用CNN实现手写数字识别
输入是[28,28,1]

conv1:[28,28,32](filter:[5,5,1],size=32,padding=same,strides=[1,1])
max_pool1:[14,14,32](ksize=[2,2],strides=[2,2])

conv2:[14,14,64](filter:[5,5,32],size=64,padding=same,strides=[1,1])
max_pool2:[7,7,64](ksize=[2,2],strides=[2,2])

flat:[7*7*64]
    w1=[7*7*64,1024]
fc1:[1024]
    w2=[1024,10]
fc2:[10]

1，数据准备

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.test.images.shape,mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


2,准备好placeholder

In [2]:
batch_size = 256
x = tf.placeholder(tf.float32,[batch_size,784],name="x")
x_image = tf.reshape(x,[-1,28,28,1])
y_ = tf.placeholder(tf.float32,[batch_size,10],name="y")

3,初始化参数/权重

In [3]:
w_conv1 = tf.Variable(tf.random_normal([5,5,1,32]),name="w_conv1")
b_conv1 = tf.Variable(tf.zeros([32]),name="b_conv1")


w_conv2 = tf.Variable(tf.random_normal([5,5,32,64]),name="w_conv2")
b_conv2 = tf.Variable(tf.zeros([64]),name="b_conv2")

w_fc1 = tf.Variable(tf.random_normal([7*7*64,1024]),name="w_fc1")
b_fc1 = tf.Variable(tf.zeros([1024]),name="b_fc1")

w_fc2 = tf.Variable(tf.random_normal([1024,10]),name="w_fc2")
b_fc2 = tf.Variable(tf.zeros([10]),name="b_fc2")


4,拿到每个类别的score

In [4]:
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image,w_conv1,strides=[1,1,1,1],padding="SAME") + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1,w_conv2,strides=[1,1,1,1],padding="SAME") + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
h_fc2 = tf.matmul(h_fc1,w_fc2)+b_fc2

logits = h_fc2

5,计算多分类的softmax的loss function

In [5]:
#交叉熵损失
entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y_,name="loss")
#求平均值
loss = tf.reduce_mean(entropy)

6,准备好optimizer

In [6]:
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

7,在session里执行graph里定义的运算

In [9]:
f = open("./_mnist_cnn.log",'a') 
n_epoch = 30
init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs/logistic_rec",sess.graph)
    sess.run(init)
    n_batch = int(mnist.train.num_examples/batch_size)
    for i in range(n_epoch):
        total_loss = 0
        for _ in range(n_batch):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={x:x_batch,y_:y_batch})
            total_loss += loss_batch
        print("epoch {0}:{1}".format(i,total_loss))
        f.write("epoch {0}:{1}".format(i,total_loss))
        f.write("\n")
    print("optimizer finished")
    f.write("optimizer finished")
    f.write("\n")
    
    #测试模型
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(y_,1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
    
    n_batch = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batch):
        x_batch,y_batch = mnist.train.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy],feed_dict={x:x_batch,y_:y_batch})
        total_correct_preds += accuracy_batch[0]
    print("accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    f.write("accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    f.close()
    writer.close()
   
        
    

epoch 0:637876.7300872803
epoch 1:23149.32145881653
epoch 2:10718.248847007751
epoch 3:5122.296646356583
epoch 4:3065.6760714659936
epoch 5:2026.704195142258
epoch 6:2271.2985424995422
epoch 7:2316.3909103538754
epoch 8:2207.657495206543
epoch 9:1601.6332616149448
epoch 10:1635.1320364382118
epoch 11:2306.3250555992126
epoch 12:2419.971252501011
epoch 13:2081.805906406602
epoch 14:1541.8159923991186
epoch 15:1791.374420169741
epoch 16:2176.9552107096715
epoch 17:2349.740435196326
epoch 18:2136.1664056777954
epoch 19:1925.0080615188926
epoch 20:1529.6951380063547
epoch 21:1941.0337077324878
epoch 22:1590.7963467091322
epoch 23:2475.137214422226
epoch 24:1931.4243476688862
epoch 25:1495.7468738276511
epoch 26:1531.10822564736
epoch 27:1534.4193782806396
epoch 28:1503.5942910909653
epoch 29:1548.0174161139876
optimizer finished
accuracy 0.9947
